In [35]:
import json
# !pip install ipywidgets
# !pip install ipysheet

In [36]:
with open("data/GiantsGatesUnitsParams.json") as datafile:
    data = json.load(datafile)

for k in data.keys():
    data[k]["type"] = k

In [37]:
import ipysheet as sheet

param_names = [
    'cost',
    'maxSpeed',
    'maxHealth',
    'attackDamage',
    'attackCooldown',
    'attackRange']
unit_names = list(data.keys())
unit_sheet = sheet.sheet(columns=len(param_names), rows=len(unit_names), row_headers=unit_names, column_headers=param_names)
for i, name in enumerate(unit_names):
    sheet.row(i, [data[name][p] for p in param_names])
unit_sheet

Sheet(cells=(Cell(column_end=5, column_start=0, row_end=0, row_start=0, squeeze_column=False, type='numeric', …